In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2019.csv", encoding='UTF-8')
print(df.shape, df.columns)
df

(164727, 3) Index(['sentence', 'predicted_sentiment', 'org_idx'], dtype='object')


,sentence,predicted_sentiment,org_idx
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,긍정,0
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,긍정,0
2,안녕하세요. 메리 성입니다,긍정,1
3,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,부정,1
4,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,긍정,1
...,...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,긍정,6126
164723,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,긍정,6126
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,부정,6126
164725,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,긍정,6126


In [16]:
for emo in ['긍정','부정','중립']:

    ''' 형태소 분석 '''
    nouns = []
    verbs = []
    adjs = []

    df_class = df[df['predicted_sentiment'] == "%s"%emo]
    corpus = df_class['sentence'].to_list()

    for sent in tqdm(range(len(corpus))):
        # 문장에서 형태소/품사 추출
        try:
            a = okt.pos(corpus[sent], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
            for x, y in a:
                # 품사가 명사면 명사 리스트에 단어 추가
                if y == 'Noun':
                    nouns.append(x)
                # 품사가 동사면 동사 리스트에 단어 추가
                elif y == 'Verb':
                    verbs.append(x)
                # 품사가 형용사면 형용사 리스트에 단어 추가
                elif y == 'Adjective':
                    adjs.append(x)
        except:
            pass

    morph_dic = {'명사':nouns, '동사':verbs, '형용사':adjs}
    for mor in ['명사','동사','형용사']:
        
        # 명사 리스트에서 명사 빈도 리스트 생성
        morph_cnt = Counter(morph_dic[mor]).most_common()

        morph = []
        morph_freq = []
        for key, val in morph_cnt:
            morph.append(key)
            morph_freq.append(val)

        # 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
        morph_df = pd.DataFrame({'morph':morph, 'morph_freq':morph_freq})
        morph_df.to_csv("2019_감정예측_%s_%s.csv"%(emo, mor), index=False, encoding='UTF-8')

100%|██████████| 184/184 [00:00<00:00, 259.98it/s]


In [17]:
import plotly.express as px

# unique 값들의 개수를 계산
sentiment_counts = df['predicted_sentiment'].value_counts()


# 막대그래프로 시각화
fig = px.bar(sentiment_counts, x=sentiment_counts.index, y=sentiment_counts.values,
             labels={'x': 'Sentiment', 'y': 'Count'},
             title='Sentiment Distribution 2019',
             color=sentiment_counts.index,
             text=sentiment_counts.values)  # 개수를 막대에 표시하기 위해 text 속성에 sentiment_counts.values 할당

# 개수를 막대 위에 표시하는 옵션 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# y축 범위 설정 (기존 최대값의 10% 더 큰 값으로 설정)
y_axis_range = sentiment_counts.max() * 1.1
fig.update_layout(yaxis_range=[0, y_axis_range])

# y축 값의 형식 설정 (정수 형식으로 표시)
fig.update_layout(yaxis_tickformat=',d')

# 글꼴 크기 설정
fig.update_layout(
    font=dict(
        family="Arial, bold",
        size=13
    )
)

# x축 레이블 설정
fig.update_xaxes(title_text='Label')

# 제목을 가운데에 위치시키기
fig.update_layout(title_x=0.5)

# 그래프 출력
fig.show()